<a href="https://colab.research.google.com/github/ssnirgudkar/UNet/blob/main/maskr_cnn_200.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. used coco mask r-cnn weights, to infer IR images - it works, gives boxes, + mask for the IR images. However, only identifies boat as that is the only class
2. Saved the unet model weights in h5 format, then used these weights instead of coco mask rcnn weights to see what happens. it doesnt work. gives some obstacles, but not correct. which means we will have to train mask r-cnn with our images - problem is they only are masked, do not have bounding boxes

In [1]:
!pip install numpy scipy
!pip install pillow scikit-image matplotlib imutils
!pip install "IPython[all]"
!pip install tensorflow-gpu # or tensorflow-gpu
!pip install keras h5py

     |████████████████████████████████| 199 kB 8.7 MB/s 
     |████████████████████████████████| 154 kB 59.5 MB/s 
     |████████████████████████████████| 454.3 MB 19 kB/s 


In [2]:
!pip install opencv-contrib-python

In [3]:
!git clone https://github.com/matterport/Mask_RCNN.git


Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 125.23 MiB | 32.38 MiB/s, done.
Resolving deltas: 100% (562/562), done.


In [4]:
cd Mask_RCNN

/content/Mask_RCNN


In [5]:
!sudo python setup.py install

/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'license-file' will not be supported in future versions. Please use the underscore name 'license_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'requirements-file' will not be supported in future versions. Please use the underscore name 'requirements_file' instead
  % (opt, underscore_opt))
running install
running bdist_egg
running egg_info
creating mask_rcnn.egg-info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
writing manifest file 'ma

In [6]:
pip show mask-rcnn

Name: mask-rcnn
Version: 2.1
Summary: Mask R-CNN for object detection and instance segmentation
Home-page: https://github.com/matterport/Mask_RCNN
Author: Matterport
Author-email: waleed.abdulla@gmail.com
License: MIT
Location: /usr/local/lib/python3.7/dist-packages/mask_rcnn-2.1-py3.7.egg
Requires: 
Required-by: 


In [7]:
!pip uninstall keras -y
!pip uninstall keras-nightly -y
!pip uninstall keras-Preprocessing -y
!pip uninstall keras-vis -y
!pip uninstall tensorflow -y
!pip uninstall h5py -y

!pip install tensorflow==1.13.1
!pip install keras==2.0.8
!pip install h5py==2.10.0

Found existing installation: Keras 2.4.3
Uninstalling Keras-2.4.3:
  Successfully uninstalled Keras-2.4.3
Found existing installation: keras-nightly 2.5.0.dev2021032900
Uninstalling keras-nightly-2.5.0.dev2021032900:
  Successfully uninstalled keras-nightly-2.5.0.dev2021032900
Found existing installation: Keras-Preprocessing 1.1.2
Uninstalling Keras-Preprocessing-1.1.2:
  Successfully uninstalled Keras-Preprocessing-1.1.2
Found existing installation: keras-vis 0.4.1
Uninstalling keras-vis-0.4.1:
  Successfully uninstalled keras-vis-0.4.1
Found existing installation: tensorflow 2.5.0
Uninstalling tensorflow-2.5.0:
  Successfully uninstalled tensorflow-2.5.0
Found existing installation: h5py 3.1.0
Uninstalling h5py-3.1.0:
  Successfully uninstalled h5py-3.1.0
     |████████████████████████████████| 92.6 MB 93.5 MB/s 
     |████████████████████████████████| 50 kB 8.1 MB/s 
     |████████████████████████████████| 367 kB 63.3 MB/s 
     |████████████████████████████████| 3.2 MB 27.8 MB/s 
 

In [ ]:
# example of inference with a pre-trained coco model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from mrcnn.visualize import display_instances
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
 
# define 81 classes that the coco model knowns about
'''
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']
'''
class_names = ['sky', 'water', 'obstacle']

# define the test configuration
class TestConfig(Config):
     NAME = "test"
     GPU_COUNT = 1
     IMAGES_PER_GPU = 1
     NUM_CLASSES = 1 + 3
 
# define the model
rcnn = MaskRCNN(mode='inference', model_dir='./', config=TestConfig())
# load coco model weights
rcnn.load_weights('/mask_rcnn_coco.h5', by_name=True)
#rcnn.load_weights('/content/drive/MyDrive/Models/h5model_unet1-0.h5', by_name=True)
# load photograph
img = load_img('/1603208989.438605_resized_color.png')
img = img_to_array(img)
# make prediction
results = rcnn.detect([img], verbose=0)
# get dictionary for first prediction
r = results[0]
# show photo with bounding boxes, masks, class labels and scores
display_instances(img, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])

In [13]:
import os
import sys
import json
import datetime
import numpy as np
from numpy import zeros, asarray
import skimage.draw

import mrcnn
import mrcnn.utils 
import mrcnn.config
import mrcnn.model

class Maritimedataset(mrcnn.utils.Dataset): 
  def load_dataset(self, dataset_dir,istrain=True):
  # Add classes. We have only one class to add.
        self.add_class("dataset", 1, "obstacle")
        self.add_class("dataset", 2, "sky")
        self.add_class("dataset", 3, "water")

        img_path = '/content/drive/MyDrive/mask-r-cnn-files/annots'
        ann_path = '/content/drive/MyDrive/mask-r-cnn-files/images' 

        self.add_image('dataset', image_id = '148201',path = img_path,annotation = ann_path)

        '''    
        images_dir = dataset_dir + '/images' 
        annotations_dir = dataset_dir + '/annots'


        for filename in os.listdir(images_dir):
          image_id = filename[:-4]
          if is_train and int(image_id) >=150:
            continue 
          if not is_train and int(image_id) <150:
            continue 

          img_path = images_dir + filename 
          ann_path = annotations_dir + image_id 
        '''
          
  def load_mask(self, image_id):
      """Generate instance masks for an image.
          Returns:
          masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
          class_ids: a 1D array of class IDs of the instance masks.
      """
      masks = cv2.imread("/content/drive/MyDrive/mask-r-cnn-files/annots/148201.png",cv2.IMREAD_UNCHANGED)
      class_ids = ['0','1','2']  
      # Return mask, and array of class IDs of each instance. Since we have
      # one class ID only, we return an array of 1s
      return masks, class_ids 

In [14]:
class MaritimeConfig(mrcnn.config.Config):
   
    # Give the configuration a recognizable name
    NAME = "Maritime_cfg"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1
    GPU_COUNT = 1

    # Number of classes (including background)
    # obstacle, sky and water
    NUM_CLASSES = 3  

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # Skip detections with < 90% confidence
    #DETECTION_MIN_CONFIDENCE = 0.9

In [16]:
train_set = Maritimedataset()
train_set.load_dataset(dataset_dir='/content/drive/MyDrive/mask-r-cnn-files', istrain=True)
train_set.prepare()

In [18]:
valid_dataset = Maritimedataset()
valid_dataset.load_dataset(dataset_dir='/content/drive/MyDrive/mask-r-cnn-files', istrain=False)
valid_dataset.prepare()



In [19]:
import mrcnn.model
model = mrcnn.model.MaskRCNN(mode='training', 
                             model_dir='./', 
                             config=MaritimeConfig())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [ ]:
model.load_weights(filepath='mask_rcnn_coco.h5', 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])